# 📝 Ejercicio Práctico de RAG: Análisis Financiero de Gigantes Tecnológicos

**¡Bienvenido/a a tu primer gran proyecto como analista de IA!**

## Misión

Tu tarea es construir un sistema de Consulta y Respuesta (Q&A) utilizando RAG para analizar y comparar los últimos informes financieros anuales (conocidos como **10-K**) de cuatro de las empresas más grandes del mundo: **Apple, Google (Alphabet), Microsoft y Meta**. 

Utilizarás las habilidades que has aprendido en los notebooks anteriores para cargar estos documentos desde la web, dividirlos en fragmentos (chunks), indexarlos en una base de datos vectorial con FAISS y construir una cadena de LangChain para hacerles preguntas complejas.

## Objetivos de Aprendizaje

1.  Aplicar el flujo de trabajo de RAG a un caso de uso del mundo real con documentos complejos.
2.  Manejar múltiples fuentes de datos y distinguirlas usando metadatos.
3.  Practicar la construcción de una cadena RAG de alto nivel con LangChain.
4.  Formular preguntas que requieran búsqueda, extracción y síntesis de información de distintos documentos.

### **Paso 0: Instalación y Configuración**

Asegúrate de tener todas las librerías necesarias y de configurar tus claves de API.

In [ ]:
# Descomenta y ejecuta si es necesario
# !pip install langchain langchain-openai langchain-google-genai langchain-community faiss-cpu beautifulsoup4

import os

# TODO: Configura tus claves de API aquí
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY"
# os.environ["COHERE_API_KEY"] = "YOUR_COHERE_API_KEY"

print("Configuración lista.")

### **Parte 1: Recolección de Datos (Código Proporcionado)**

Te proporcionamos los enlaces a los últimos informes 10-K de cada empresa, directamente desde la base de datos EDGAR de la SEC (Comisión de Bolsa y Valores de EE. UU.).

Usaremos `WebBaseLoader` de LangChain para cargar el contenido de estas URLs. Un paso **crucial** que hemos añadido es un bucle para añadir el nombre de la empresa a los **metadatos** de cada documento cargado. Esto será fundamental más adelante para saber de qué empresa proviene cada fragmento de información.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

urls = {
    "apple": "https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm",
    "google": "https://www.sec.gov/Archives/edgar/data/1652044/000165204424000022/goog-20231231.htm",
    "microsoft": "https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm",
    "meta": "https://www.sec.gov/Archives/edgar/data/1326801/000132680124000012/meta-20231231.htm"
}

all_documents = []
print("Cargando documentos desde la web... Esto puede tardar un momento.")

for company, url in urls.items():
    print(f"Cargando informe de {company.capitalize()}...")
    loader = WebBaseLoader(url)
    # Cargamos los documentos de la URL
    company_docs = loader.load()
    # Añadimos el nombre de la empresa a los metadatos de cada documento
    for doc in company_docs:
        doc.metadata['company'] = company
    all_documents.extend(company_docs)

print(f"\nCarga completa. Total de documentos cargados: {len(all_documents)}")
print("Ejemplo de metadatos del primer documento:", all_documents[0].metadata)

### **Parte 2: Chunking de los Documentos (¡Tu Turno!)**

Los informes 10-K son extremadamente largos. Necesitas dividirlos en chunks manejables. Usa `RecursiveCharacterTextSplitter` para esta tarea. Elige un tamaño de chunk y un solapamiento que consideres adecuados para textos densos como estos.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# TODO: Instancia el RecursiveCharacterTextSplitter.
# Pista: Un chunk_size entre 1500 y 2500 y un chunk_overlap de 100-200 suele funcionar bien.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=200
)

# TODO: Divide los `all_documents` en `all_chunks`.
all_chunks = text_splitter.split_documents(all_documents)

print(f"Documentos divididos en {len(all_chunks)} chunks.")
print("\n--- Ejemplo de un Chunk ---")
print("Contenido:", all_chunks[100].page_content[:300] + "...")
print("Metadatos:", all_chunks[100].metadata)

### **Parte 3: Creación de la Base de Datos Vectorial (¡Tu Turno!)**

Ahora, convierte tus chunks en vectores y almacénalos en una base de datos FAISS. 

1.  Elige e instancia un modelo de embeddings (`OpenAIEmbeddings`, `GoogleGenerativeAIEmbeddings`, etc.).
2.  Usa `FAISS.from_documents()` para crear el índice vectorial. ¡Prepárate, este paso consumirá llamadas a la API y puede tardar unos minutos!

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# O puedes usar: from langchain_google_genai import GoogleGenerativeAIEmbeddings

# TODO: 1. Instancia tu modelo de embeddings preferido.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

print("Generando embeddings y creando el vector store FAISS... Paciencia.")
# TODO: 2. Crea el vector store FAISS a partir de `all_chunks` y el modelo de `embeddings`.
vectorstore = FAISS.from_documents(all_chunks, embeddings)

print("\n¡Vector store creado con éxito!")

### **Parte 4: Construcción de la Cadena de RAG (¡Tu Turno!)**

Es hora de ensamblar la cadena que responderá a nuestras preguntas. 

1.  Crea un `retriever` a partir del `vectorstore`.
2.  Elige un `ChatModel` para la generación de la respuesta.
3.  Define un `prompt` (puedes usar `hub.pull("rlm/rag-prompt")` o crear el tuyo).
4.  Une todo usando el LangChain Expression Language (LCEL).

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

# TODO: 1. Crea un retriever a partir del vectorstore. 
# Pista: .as_retriever() puede tomar argumentos como search_kwargs={"k": 3} para definir cuántos chunks recuperar.
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# TODO: 2. Instancia tu LLM preferido.
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. Un prompt genérico para RAG.
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(f"Fuente: {doc.metadata.get('company', 'desconocida')}\n{doc.page_content}" for doc in docs)

# TODO: 4. Define la cadena RAG usando LCEL. 
# La estructura `{"context": retriever | format_docs, "question": RunnablePassthrough()}` es un excelente punto de partida.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Cadena RAG lista para recibir preguntas.")

### **Parte 5: ¡A Preguntar! El Momento de la Verdad**

Usa la `rag_chain` que has construido para responder a las siguientes preguntas. Analiza las respuestas y piensa si son correctas y completas.

**Consejo:** Si una respuesta no es buena, ¡intenta reformular la pregunta! La forma en que preguntas (Prompt Engineering) es tan importante como el sistema RAG en sí.

In [ ]:
pregunta_1 = "Resume los principales factores de riesgo de negocio mencionados por Meta en su informe."
print(f'--- Pregunta: {pregunta_1} ---')
# TODO: Invoca la cadena con la pregunta 1
# respuesta_1 = rag_chain.invoke(pregunta_1)
# print(respuesta_1)


In [ ]:
pregunta_2 = "¿Cuál fue el total de ingresos (Total Revenues) de Apple y Microsoft en su último año fiscal reportado? Cita las cifras y compáralas."
print(f'--- Pregunta: {pregunta_2} ---')
# TODO: Invoca la cadena con la pregunta 2
# respuesta_2 = rag_chain.invoke(pregunta_2)
# print(respuesta_2)

In [ ]:
pregunta_3 = "Busca la frase 'Artificial Intelligence' en los documentos de Google y Microsoft. ¿Qué puedes inferir sobre la importancia que le dan a la IA basándote en el contexto donde aparece?"
print(f'--- Pregunta: {pregunta_3} ---')
# TODO: Invoca la cadena con la pregunta 3
# respuesta_3 = rag_chain.invoke(pregunta_3)
# print(respuesta_3)

In [ ]:
pregunta_desafio = "Basándote en las secciones de 'Risk Factors', ¿qué riesgo común relacionado con la ciberseguridad o la privacidad de datos mencionan tanto Apple como Meta?"
print(f'--- Pregunta de Desafío: {pregunta_desafio} ---')
# TODO: Invoca la cadena con la pregunta de desafío
# respuesta_desafio = rag_chain.invoke(pregunta_desafio)
# print(respuesta_desafio)

## Conclusión y Pistas Finales

¡Enhorabuena! Has completado un proyecto de RAG de principio a fin sobre un caso de uso complejo y realista. 

**Para reflexionar:**
* ¿Fueron las respuestas siempre correctas? ¿En qué casos falló el sistema?
* ¿Cómo podría mejorarse el sistema? (¿Chunks más grandes/pequeños? ¿Otro modelo de embedding? ¿Un prompt más detallado?).
* La función `format_docs` que te dimos incluye el nombre de la empresa. ¿Cómo te ayudó esto a verificar las respuestas del LLM?

Este ejercicio demuestra que la potencia de los LLMs se multiplica cuando los conectamos a datos específicos y relevantes. ¡Ahora tienes la base para construir aplicaciones de IA mucho más potentes!